In [1]:
import os
import pandas as pd
import datetime

# Data List

In [2]:
root_dir = os.getcwd()
f_lists = os.listdir(root_dir)
new_file_lists = [f for f in f_lists if f.endswith('.csv')]
data_lists = new_file_lists[:-1]

# Make Function

In [3]:
def csv_read_(data_dir, data_list):
    tmp = pd.read_csv(os.path.join(data_dir, data_list), sep=',', encoding='utf-8')
    y, m, d = map(int, data_list.split('-')[-1].split('.')[:-1])
    time = tmp['Time']
    tmp['DTime'] = '-'.join(data_list.split('-')[-1].split('.')[:-1])
    ctime = time.apply(lambda _ : _.replace(u'오후', 'PM').replace(u'오전', 'AM'))
    n_time = ctime.apply(lambda _ : datetime.datetime.strptime(_, "%p %I:%M:%S"))
    newtime = n_time.apply(lambda _ : _.replace(year=y, month=m, day=d))
    tmp['Time'] = newtime
    return tmp

# Make Dedicated DataFrame

In [4]:
dd = csv_read_(root_dir, data_lists[0])
for i in range(1, len(data_lists)):
    dd = pd.merge(dd, csv_read_(root_dir, data_lists[i]), how='outer')

In [5]:
dd = dd.drop('Index', axis=1)
dd = dd.set_index('Time')

In [6]:
dedicated_data = dd.copy()
dedicated_data = dedicated_data.dropna()
dedicated_data

,LoT,pH,Temp,DTime
Time,,,,
2021-09-06 09:01:18,1,1.02,47.18,2021-09-06
2021-09-06 09:01:23,1,1.05,47.34,2021-09-06
2021-09-06 09:01:28,1,1.09,48.45,2021-09-06
2021-09-06 09:01:33,1,1.12,48.46,2021-09-06
2021-09-06 09:01:38,1,1.15,48.47,2021-09-06
...,...,...,...,...
2021-10-27 11:14:41,22,2.79,51.83,2021-10-27
2021-10-27 11:14:46,22,3.62,42.20,2021-10-27
2021-10-27 11:14:51,22,3.40,41.88,2021-10-27


In [7]:
perc =30
data_columns = ['pH', 'Temp']
for df_name in list(data_columns):
    print(f'DataFrame name: {df_name}')
    df = dedicated_data[df_name]
    print('[step 1-1]')
    print(round(df.isnull().sum()/len(df)*100,2))
    print('[step 1-2]')
    print(df.isnull().sum()/len(df)*100>perc)
    print('[step 2-1]')
    print(df.isnull().head())
    print('[step 2-2]')
    print(df.isnull().sum())
    print('[step 2-3]')
    cmpt_len =df.isnull().sum().sum()
    print(cmpt_len)
    print("결측치 = %d개 \n완전성 지수 : %.2f%% "%(cmpt_len,(1-cmpt_len/len(df))*100))
    print('\n\n')

DataFrame name: pH
[step 1-1]
0.0
[step 1-2]
False
[step 2-1]
Time
2021-09-06 09:01:18    False
2021-09-06 09:01:23    False
2021-09-06 09:01:28    False
2021-09-06 09:01:33    False
2021-09-06 09:01:38    False
Name: pH, dtype: bool
[step 2-2]
0
[step 2-3]
0
결측치 = 0개 
완전성 지수 : 100.00% 



DataFrame name: Temp
[step 1-1]
0.0
[step 1-2]
False
[step 2-1]
Time
2021-09-06 09:01:18    False
2021-09-06 09:01:23    False
2021-09-06 09:01:28    False
2021-09-06 09:01:33    False
2021-09-06 09:01:38    False
Name: Temp, dtype: bool
[step 2-2]
0
[step 2-3]
0
결측치 = 0개 
완전성 지수 : 100.00% 





In [8]:
check_unique =dedicated_data[
    ['pH','Temp']
    ].value_counts().reset_index()
perc_check_unique_item_urgent_info =round(
    (len(check_unique)-len(check_unique[check_unique[0]>1]))
    /len(check_unique)*100,2)

print(f'The percentage of uniqueness for pH<->Temp : {perc_check_unique_item_urgent_info}')
print("유일성 지수 : %.2f%% "%(perc_check_unique_item_urgent_info))

The percentage of uniqueness for pH<->Temp : 37.55
유일성 지수 : 37.55% 


In [9]:
df = dedicated_data
pH_lb =df['pH']>=1
pH_ub =df['pH']<=4
temp_lb =df['Temp']>=35
temp_ub =df['Temp']<=65
vald_df =df[pH_lb &pH_ub &temp_lb &temp_ub]
print(f'[Step 1] 데이터 범위를 벗어난 데이터 수: {len(df)-len(vald_df)}')

d0 = '2021-09-06'
d1 = '2021-10-27'
vald_df =vald_df.loc[d0:d1]
print(f'[Step 2] 수집된 날짜를 벗어나는 데이터 수: {len(df)-len(vald_df)}')

vald_df['DTime']=vald_df['DTime'].apply(lambda x:isinstance(x,datetime.datetime))
vald_df['pH']=vald_df['pH'].apply(lambda x:isinstance(x,float))
vald_df['Temp']=vald_df['Temp'].apply(lambda x:isinstance(x,float))
print(f'[Step 3] 데이터 형식을 벗어나는 데이터 수: {len(df)-len(vald_df)}')

vald_df[
    (vald_df['pH']==True)
    &(vald_df['Temp']==True)
    ]
vald_len =len(vald_df)
item_vald =vald_len/len(df)*100
print("유효성 지수 : %.2f%% "%(item_vald)) 

[Step 1] 데이터 범위를 벗어난 데이터 수: 0
[Step 2] 수집된 날짜를 벗어나는 데이터 수: 0
[Step 3] 데이터 형식을 벗어나는 데이터 수: 0
유효성 지수 : 100.00% 


C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_21956/2132131785.py:11: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  vald_df =vald_df.loc[d0:d1]
